## The Impact of Intermediate Dimensionality on the Clustering Coherence of News Embeddings using HDBSCAN
---
This notebook evaluates clustering pipelines for news article embeddings by systematically varying intermediate dimensionality reduction and measuring cluster coherence, coverage, and stability. We show that clustering quality is highly sensitive to the reduced dimension and that stable, interpretable trends emerge only within a narrow range of pipeline configurations.

In [1]:
import pandas as pd

### Introduction
---
Recent progress in transformer-based language models has made it straightforward to represent large collections of text as high-dimensional semantic embeddings. Because these embeddings are often massive (over 3,000 dimensions), we rarely cluster them directly. Instead, we use "intermediate" steps to shrink the data down before grouping it.

Why do we do this? Direct clustering in high dimension is hindered by _distance concentration_. Geometrically, as dimensionality increases, the contrast between the distance to the nearest and farthest neighbors diminishes. Density-based algorithms like [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html) identifies clusters by evaluating the distance and connectivity of a Minimum Spanning Tree. This 'flatness' in distance prevents the algorithm from distinguishing between dense topical cores and the sparse background noise. Intermediate dimensionality reduction serves to __restore the local density contrast__ necessary for HDBSCAN’s reachability metrics to function.

In most data pipelines, developers use Principal Component Analysis (PCA) or Uniform Manifold Approximation and Projection (UMAP) to handle this shrinkage. While PCA is a classic, linear approach that focuses on broad patterns, UMAP is a modern, non-linear method designed to keep related items close together. However, after searching for a 'best' intermediate dimension, most sources pointed to heuristics.

This choice is especially important for Hierarchical Density-Based Spatial Clustering of Applications with Noise(HDBSCAN), a clustering algorithm that groups data based on how "dense" it is. Because the very concept of "density" changes depending on how many dimensions you have, the intermediate step isn't just a technicality—it might completely change which news stories get grouped together and which are thrown out as "noise."

### The Goal
---
I'm particularly intrested in using HDBSCAN for automating trend detection in news for a digestive reading app called SightRead. Through this project I hope to learn more about the tradeoffs in choosing a clustering pipeline, and simply document findings. 

In this work, I explore how changing the number of intermediate dimensions for both PCA and UMAP affects the final quality of news clusters. By testing a range of dimensions, I aim to measure:

__Cluster Coherence__: How semantically similar are the articles within a single group?

__Noise Levels__: How much of our data is discarded as "un-clusterable" in different dimensions?

__Method Comparison__: Does UMAP’s non-linear approach actually produce better clusters for news than the simpler PCA method?

By analyzing embeddings from RSS news feeds, I hope to provide a practical guide for which dimensionality reduction settings actually work best for organizing the daily news.

### An Overview of Data Collection
---
I'll give a quick overview of where and how the data is collected, more information on the dataset can be found [here](https://github.com/evansun06/trend-analysis-sr/blob/main/wrangling.ipynb). Articles were polled for a 1-week window from January 5th to January 12th from a select subset of RSS feeds from _The Guardian_, _The Wall Street Journal_, _United Nations_, _Fox News_, and _Daily Mail_.

<p align="center">
	<img src="assets/rss_aggregation.png" alt="Data aggregation diagram" width="720" />
</p>

_figure: SightRead RSS aggregation workflow_



### Methodology Part I: Effect of Intermediate Dimensionality on the Cluster Quality of HDBSCAN with respect to PCA and UMAP
---

__Pipeline configurations__ $(d, {mcs}, {ms})$:
- intermediate dimension $d \in \{2, 5, 10, 15, 25, 50, 75, 100\}$, we run the following pipelines.
- min_cluster_size(mcs): HDBSCAN hyperparameter $\{5, 10, 15, 30\}$
- min_samples(ms): HDBSCAN hyperparameter $\{5, 10, 15\}$

__Data and Embeddings__

Let $N$ be the number of news articles contained in the dataset. Each article $i$ is represented by an embedding vector

$$
x_i \in \mathbb{R}^D
$$

where D is the embedding dimension (3072 in our case).

We stack embeddings into a matrix $X \in \mathbb{R}^{D\times N}$ where

$$

X = \begin{bmatrix} x_1^{\top} \\ x_2^{\top} \\ \vdots \\ x_N^{\top}\end{bmatrix} \in \mathbb{R}^{N \times D}

$$

For each pipeline configuration we run both clustering pipelines:

__A. Principle Component Analysis__

1. L2 Normalization (scikit-learn)
$$
\forall x_i \in X \rightarrow \hat{x}_i = \frac{x_i}{||x_i||_2}
$$
2. Reduce to intermediate dimension via PCA
$$
    {PCA}_{(d)}: \mathbb{R}^D \rightarrow \mathbb{R}^d
$$

$$
    X \rightarrow Z^{(pca)}_d,\ \ \  Z^{(pca)}_d\in \mathbb{R}^{N \times d}
$$

3. Cluster using HDBSCAN with variated hyperparameters
    - `min_cluster_size`
    - `min_samples`
    - `output`: Assigns each article $i$ with label $\ell_i \in \{-1, 0, 1, ..., K-1\}$ where -1 is noise

__B. Unifold Manifold Approximation and Projection__
- Repeat with UMAP
$$
{UMAP}_{(d)}: \mathbb{R}^D \rightarrow \mathbb{R}^d
$$

$$
    X \rightarrow Z^{(umap)}_d,\ \ \  Z^{(umap)}_d\in \mathbb{R}^{N \times d}
$$

### Metrics 
After performing clustering for all tuples(d, mcs, ms) for both piplines we select the optimized pipeline configuration for both UMAP and PCA via these metrics below.

Let $K$ denote the number of clusters discovered by HDBSCAN for a given run, excluding noise. We index clusters by $k \in \{0, 1, ..., K - 1\}$, with noise labeled as $-1$.


__1.  Cluster coherence__

Cluster to centroid cosine similarity. Represents the semantic coherence of a cluster

For each cluster $k \in \{0, ..., K-1\}$, let $S_k$ be the set of all articles $i$ grouped within cluster $k$, and $n_k = |S_k|$

__1.1__ Compute the centroid (mean embedding):

$$
    \mu_k = \frac{1}{n_k}\sum_{i \in S_k}{\hat{x}_i}

$$

__1.2__ Normalize the centroid

$$
    \hat{\mu}_k = \frac{\mu_k}{||\mu_k||_2}

$$

__1.3__ Cluster coherence score

For each article $i$ in the cluster set, $S_k$, we calculate the cosine simlarity to the centroid $\hat{\mu}_k$, and average.

$$
    \text{coh}(k) = \frac{1}{n_k} \sum_{i\in S_k} \hat{x}_i^\top \hat{\mu}_k
$$

__1.4__ Size-weighted overall coherence

Primary metric for pipeline cluster coherence, weighted to counteract disproportional clusters.

$$
    \text{coh}_{weighted} = \frac{\sum^{K-1}_{k=0}{n_k \text{coh}(k)}}{\sum^{K-1}_{k=0}{n_k}}
$$




__2. Cluster and Noise Fraction__

Let the set of clustered points be 

$$
\mathcal{C} = \{i | \ell_i \neq -1 \}, \ \ |\mathcal{C}| = N_c
$$

let the set of noise points be 

$$
\mathcal{N} = \{i | \ell_i = -1\},\ \  |\mathcal{N}| = N_n
$$

and $N = N_c + N_n$.

Then we define the Noise Fraction as `noise_frac` = $\frac{N_n}{N}$ and Cluster Fraction as `cluster_frac` = $\frac{N_c}{N}$


__3. Sanity Metrics__

Avoids trivial cases
1.  Number of clusters: $K$
2.  Largest cluster fraction:
$$
\frac{max_k n_k}{N}
$$

### Methodology Part II: Comparative Pipeline Evaluation and Stability Analysis
---

__1. Cross-Pipeline Evaluation__

To determine the superior dimensionality reduction strategy, we compare the optimized PCA and UMAP pipelines. This comparison weighs the trade-off between internal semantic density and topological robustness.
    - __Relative Coherence Gain__: The percentage difference in $\text{coh}_{weighted}$ between PCA and UMAP.
    - __Granularity__: A comparison of $K_{pca}$ versus $K_{umap}$ to identify which method captures a more nuanced topic hierarchy.
    - __Case-by-case observations__: View select clusters by hand and document observervations.

__2. Stability Analysis via Subsampling__

To ensure the discovered clusters represent robust semantic structures rather than stochastic one-offs, we perform a stability stress test.

__2.1__ Subsampling Procedure

We perform $M$ iterations of subsampling without replacement.

- In each iteration $m$, a subset $I_m \subset I$ is drawn such that $|I_m| = 0.8N$.
- We then run the optimized PCA and UMAP pipelines for each iteration.

*_Subsampling without replacement is critical to prevent "artificial coherence" caused by identical vector duplicates, which would otherwise bias the cosine similarity scores._

__2.2__ Consistency Metrics

For each iteration $m$, we re-calculate and track metrics like _relative coherence gain_, $K$, $\text{coh}_{weighted}$. After all M iterations we observe:
    
- __Cluster Count Variance__ ($\sigma^2_K$): Measures the decisiveness of the pipeline.
$$
\sigma^2_K = \frac{1}{M-1} \sum_{m=1}^{M} (K_m - \bar{K})^2
$$

- __Coherence Stability__: The standard deviation of $\text{coh}_{weighted}$ across runs, indicating if the "thematic tightness" is dependent on specific articles.



$$
\sigma_{\text{coh}_{weighted}} = \sqrt{\frac{1}{M-1} \sum_{m=1}^{M} ({\text{coh}_{weighted}}_m - {\text{coh}_{weighted}}_{avg})^2}
$$

- __Adjusted Rand Index (ARI)__: The primary measure of __Label Persistence__. For the set of points present in both the original run (labels $L_{orig}$) and the subsampled run (labels $L_m$), we calculate:

$$ARI(L_{orig}, L_m) = \frac{\sum_{ij} \binom{n_{ij}}{2} - [\sum_i \binom{a_i}{2} \sum_j \binom{b_j}{2}] / \binom{n}{2}}{\frac{1}{2} [\sum_i \binom{a_i}{2} + \sum_j \binom{b_j}{2}] - [\sum_i \binom{a_i}{2} \sum_j \binom{b_j}{2}] / \binom{n}{2}}$$
- An $ARI \approx 1.0$ indicates perfect cluster survival.
- An $ARI \approx 0$ indicates that the clusters are essentially random and do not persist across data perturbations.

### Dataset overview

__Fields__ (post-wrangling):
1. article_id: UUID for a unique article.
2. ingestion_id: UUID for the ingested article (articles can be ingested in different ways, in this dataset all used the `default` version).
3. normalized_text: Headline concatenated with description, normalized.
4. polled_at: timestamp when the article was polled at (each RSS feed was polled by the hour)
5. published_at: the publication date of the article for possible time series analysis
6. embedding: the embedding vector (OpenAI text-embedding-3-large)

*_see `wrangling.ipynb` for wrangling procedures_

In [2]:
df = pd.read_csv("data/wrangled_articles.csv")
df.head()

,article_id,ingestion_id,normalized_text,polled_at,published_at,embedding
0,d9509dc9-a405-4866-9201-19e3d063d137,000d8dea-8f60-470e-b539-eb51780b55db,Outrage as 'miserable' shovel-wielding neighbo...,2026-01-07 13:33:24.714064+00,2026-01-07 13:29:24+00,[-0.01143522 -0.03746951 -0.00753336 ... -0.00...
1,9a156ce2-11d6-4eff-b243-e2d2bb208faa,0072ab60-b1ed-42a9-9624-88c18292e588,"Morality, military might and a sense of mischi...",2026-01-09 05:17:56.535446+00,2026-01-09 03:15:08+00,[ 0.00615122 -0.03799464 -0.01626345 ... 0.00...
2,df3b2fc9-4d0f-4199-bca1-ea6d954e0c31,007f72ee-5039-47bb-8c61-c981f2810d3a,Republican senator vows to block all Fed nomin...,2026-01-12 15:17:02.967673+00,2026-01-12 14:55:53+00,[-0.00727808 -0.02314259 -0.02138059 ... 0.01...
3,d0f6077e-15be-497e-9a2b-1c023486ce9a,00879af4-8278-40a8-8338-dba8a0b45810,Starmer prepares to rip up Brexit: PM ready to...,2026-01-05 04:42:39.19293+00,2026-01-04 15:49:23+00,[ 0.01808163 0.02099675 -0.01054433 ... 0.00...
4,847a6cef-0c95-4a0f-9388-7345ba01bf26,00937788-6ba8-4db8-8107-645754dd7362,ANOTHER poll shows Labour in third behind Refo...,2026-01-07 11:12:56.5161+00,2026-01-07 10:57:35+00,[ 0.03266984 -0.00281101 -0.01202355 ... 0.01...


### References